# Frustrated Ring Model

https://doi.org/10.1088/2058-9565/acfbaa

In [ ]:
using QAOA, Distributions, Interpolations, LinearAlgebra, Arpack
using HDF5, Printf
using PythonPlot
PythonPlot.matplotlib.style.use("./paper.mplstyle")
PythonPlot.rc("axes", prop_cycle = PythonPlot.matplotlib.cycler(color = ["#2D5FAA", "#B7293F", "#438E6A", "#F7BD2E", "#F16C37"]))

PATH = "/home/ubuntu/Archives/"
PLOT_PATH = "/home/ubuntu/Archives/plots/SK_model/paper/";

In [ ]:
using Revise, SpinFluctuations

In [ ]:
Base.show(io::IO, f::Float64) = @printf(io, "%1.4f", f)

In [ ]:
all_data = Dict();

In [ ]:
# N = 11
N = 13
# N = 15
N = 17
# N = 19
N = 101
@assert N % 2 != 0
nev=32
keep_EVs = 4;

In [ ]:
data_N = Dict();

In [ ]:
J = 1.
J_L = 0.5
J_R = 0.45;

In [ ]:
J_mat = zeros(N, N)
for n in 1:(N - 1) ÷ 2
    J_mat[n, n + 1] = J
end
J_mat[(N - 1) ÷ 2, (N + 1) ÷ 2] = J_L 
J_mat[(N + 1) ÷ 2, (N + 3) ÷ 2] = J_L  
for n in (N + 3) ÷ 2:N-1
    J_mat[n, n + 1] = J
end
J_mat[1, N] = -J_R
J_mat = J_mat + transpose(J_mat);

In [ ]:
mf_problem = Problem(0, J_mat);
full_problem = Problem(0, zeros(size(J_mat)[1]), J_mat)
fix_idx = (N + 1) ÷ 2
smart_problem = Problem(0, J_mat[1:end .!= fix_idx, fix_idx], J_mat[1:end .!= fix_idx, 1:end .!= fix_idx]);

In [ ]:
# T_final = 2.0^15
T_final = 2.0^16
# npts = 2048
npts = 4096
coarse_times = range(0, 1, npts + 1);
exact_times = range(0, 1, 129);

## Exact gap

In [ ]:
# fixing spin
eigeninfo = map(s -> (eigs(-SpinFluctuations.hamiltonian(1 - s, s, smart_problem.local_fields, smart_problem.couplings), nev=nev, which=:LM, maxiter=10000)), exact_times)
H_x = SpinFluctuations.hamiltonian(1, 0, smart_problem.local_fields, smart_problem.couplings)
H_z = SpinFluctuations.hamiltonian(0, 1, smart_problem.local_fields, smart_problem.couplings)
all_eigvecs = zeros(length(exact_times), 2^(N-1), keep_EVs)

λs = [vals[1] for vals in eigeninfo]
λ = sort(reduce(hcat, λs), dims=1)

gap = λ[2, :] .- λ[1, :];

In [ ]:
# not fixing spin
eigeninfo = map(s -> (eigs(-SpinFluctuations.hamiltonian(1 - s, s, full_problem.local_fields, full_problem.couplings), nev=nev, which=:LM, maxiter=10000)), exact_times)
all_eigvecs = zeros(length(exact_times), 2^(N), keep_EVs)
H_x = SpinFluctuations.hamiltonian(1, 0, full_problem.local_fields, full_problem.couplings)
H_z = SpinFluctuations.hamiltonian(0, 1, full_problem.local_fields, full_problem.couplings)

λs = [vals[1] for vals in eigeninfo]
λ = sort(reduce(hcat, λs), dims=1)

gap = λ[3, :] .- λ[1, :];

In [ ]:
for k in 1:length(exact_times)
    sorting_perm = sortperm(λs[k])
    all_eigvecs[k, :, :] .= eigeninfo[k][2][:, sorting_perm[1:keep_EVs]]
end

In [ ]:
mingap = minimum(gap)
mingap |> println
gap_idx = findfirst(x -> x == mingap, gap)
gaploc = exact_times[gap_idx]
gaploc |> println

In [ ]:
gs = [all_eigvecs[k, :, 1] for k in 1:length(exact_times)]
first_ex = [all_eigvecs[k, :, 2] for k in 1:length(exact_times)]
second_ex = [all_eigvecs[k, :, 3] for k in 1:length(exact_times)]
third_ex = [all_eigvecs[k, :, 4] for k in 1:length(exact_times)]

overlap_01_x = [first_ex[k]' * H_x * gs[k] for k in 1:length(exact_times)]
overlap_01_z = [first_ex[k]' * H_z * gs[k] for k in 1:length(exact_times)]

overlap_02_x = [second_ex[k]' * H_x * gs[k] for k in 1:length(exact_times)]
overlap_02_z = [second_ex[k]' * H_z * gs[k] for k in 1:length(exact_times)]

overlap_03_x = [third_ex[k]' * H_x * gs[k] for k in 1:length(exact_times)]
overlap_03_z = [third_ex[k]' * H_z * gs[k] for k in 1:length(exact_times)]

overlap_12_x = [second_ex[k]' * H_x * first_ex[k] for k in 1:length(exact_times)]
overlap_12_z = [second_ex[k]' * H_z * first_ex[k] for k in 1:length(exact_times)]

overlap_13_x = [third_ex[k]' * H_x * first_ex[k] for k in 1:length(exact_times)]
overlap_13_z = [third_ex[k]' * H_z * first_ex[k] for k in 1:length(exact_times)];

In [ ]:
data_N["eigvals"] = [λ[i, :] for i in 1:4]
data_N["eigvecs"] = [all_eigvecs[i, :, :] for i in 1:4];
data_N["overlaps"] = [abs.(overlap_01_z .- overlap_01_x), abs.(overlap_02_z .- overlap_02_x), abs.(overlap_03_z .- overlap_03_x)]
data_N["g_alphas"] = [abs.(overlap_01_z .- overlap_01_x) ./ ((λ[2, :] .- λ[1, :]).^2), abs.(overlap_02_z .- overlap_02_x) ./ ((λ[3, :] .- λ[1, :]).^2)];

In [ ]:
all_data[N] = data_N;

In [ ]:
all_data[N]["g_alphas"]

#### Fixing spin

In [ ]:
figure(figsize = (4, 4))

ax = subplot(211)
axvline(gaploc, ls = "-", c="k", alpha=0.2, lw=6)
for i in 1:10
	if i == 2 
		plot(exact_times, (λ[i, :] .- λ[1, :]), lw = 2.0, alpha = 1)
	elseif i == 3		
		plot(exact_times, (λ[i, :] .- λ[1, :]), lw = 1.25, alpha = 1)	
	else
		plot(exact_times, (λ[i, :] .- λ[1, :]), "-k", lw=0.75, alpha = 1)
	end
end
plot([], [], "-k", lw = 1, label = @sprintf("\$N=%i\$", N-1))
xlim(0.0, 1.0)
ax.set_yticks(2 .* [0, 0.5, 1.0])
ylim(0, 2)
ax.set_xticklabels([])
ylabel("\$E_\\alpha - E_0\$")
legend(frameon = false, handlelength=1., handletextpad=0.5, borderaxespad=0.2)

ax = subplot(212)
axvline(gaploc, ls = "-", c="k", alpha=0.2, lw=6)
ax.semilogy(exact_times, abs.(overlap_02_z .- overlap_02_x) ./ ((λ[3, :] .- λ[1, :]).^2), label="\$\\alpha = 2\$", "-C1", lw=1.25)
ax.semilogy(exact_times, abs.(overlap_01_z .- overlap_01_x) ./ ((λ[2, :] .- λ[1, :]).^2), label="\$\\alpha = 1\$", "-C0", lw=2.0)

ax.set_xlim(0.0, 1.0)
# ax.set_ylim(1e-1, 1e5)
ax.set_ylim(1e-1, 1e2)
# ax.set_ylim(0, 50)
# ax.set_yticks([0, 25, 50])
ax.set_xlabel("\$s\$")
ax.set_ylabel("\$ g_\\alpha(s)\$")
ax.legend(loc="upper left", handlelength=1.2, handletextpad=0.5, borderaxespad=0.3)

tight_layout(pad=0.25)
savefig(PLOT_PATH * @sprintf("frustrated_ring_exact_spectrum_N_%i.pdf", N-1), dpi=256, bbox_inches="tight")
display(gcf())
PythonPlot.close();

#### Not fixing spin

In [ ]:
figure(figsize = (4, 4))

ax = subplot(211)
axvline(gaploc, ls = "-", c="k", alpha=0.2, lw=6)
for i in 1:10
	if i == 3 
		plot(exact_times, (λ[i, :] .- λ[1, :]), lw = 2.0, alpha = 1)
	elseif i == 4		
		plot(exact_times, (λ[i, :] .- λ[1, :]), lw = 1.25, alpha = 1)
	else
		plot(exact_times, (λ[i, :] .- λ[1, :]), "-k", lw=0.75, alpha = 1)
	end
end
plot([], [], "-k", lw = 1, label = @sprintf("\$\\mathcal{N}=%i\$", N))
xlim(0.0, 1.0)
ax.set_yticks(2 .* [0, 0.5, 1.0])
ylim(0, 2)
ax.set_xticklabels([])
ylabel("\$E_\\alpha - E_0\$")
legend(frameon = false, handlelength=1., handletextpad=0.5, borderaxespad=0.2)

ax = subplot(212)
axvline(gaploc, ls = "-", c="k", alpha=0.2, lw=6)
ax.semilogy(exact_times, abs.(overlap_03_z .- overlap_03_x) ./ ((λ[4, :] .- λ[1, :]).^2), label="\$\\alpha = 3\$", "-C1", lw=1.25)
ax.semilogy(exact_times, abs.(overlap_02_z .- overlap_02_x) ./ ((λ[3, :] .- λ[1, :]).^2), label="\$\\alpha = 2\$", "-C0", lw=2.0)

# ax.plot(exact_times, abs.(overlap_13_z .- overlap_13_x) ./ ((λ[4, :] .- λ[1, :]).^2), "--C1", lw=1.25)
# ax.plot(exact_times, abs.(overlap_12_z .- overlap_12_x) ./ ((λ[3, :] .- λ[1, :]).^2), "--C0", lw=1.25)

ax.set_xlim(0.0, 1.0)
ax.set_ylim(1e1, 1e4)
# ax.set_yticks([1e0, 1e2, 1e4])
ax.set_xlabel("\$s\$")
ax.set_ylabel("\$ g_\\alpha(s)\$")
ax.legend(loc="upper left", handlelength=1.2, handletextpad=0.5, borderaxespad=0.3)

tight_layout(pad=0.25)
savefig(PLOT_PATH * @sprintf("frustrated_ring_exact_spectrum_N_%i.pdf", N), dpi=256, bbox_inches="tight")
display(gcf())
PythonPlot.close();

## Mean-field trajectories

In [ ]:
tol = 1e-6 
tol = 1e-8
schedule(t) = t / T_final
sol = evolve_mean_field(mf_problem.local_fields, mf_problem.couplings, T_final, schedule, rtol=1e2tol, atol=tol) 

# get mean-field solution
solution = S -> sign.([S[3, i] for i in 1:size(S)[2]])
mf_sol = solution(sol(T_final));

### Bloch coordinates, Magnetization & Complex Coordinates

In [ ]:
sol_u = zeros(length(sol.u), size(sol.u[1])...)
for i in 1:length(sol.u)
    sol_u[i, :, :] .= sol.u[i]
end

In [ ]:
nx_vals = n_vals("x", sol_u)
ny_vals = n_vals("y", sol_u)
nz_vals = n_vals("z", sol_u);

In [ ]:
sol_t = sol.t
nx_coarse = n_coarse(nx_vals, sol_t, coarse_times)
ny_coarse = n_coarse(ny_vals, sol_t, coarse_times)
nz_coarse = n_coarse(nz_vals, sol_t, coarse_times);

In [ ]:
S_vals = [transpose(reduce(hcat, [nx_coarse[:, k], ny_coarse[:, k], nz_coarse[:, k]])) |> Matrix for k in 1:npts+1]
magnetizations = reduce(hcat, map(S -> magnetization(S, mf_problem.local_fields, mf_problem.couplings), S_vals));

In [ ]:
areas = Dict()
EA_param = Dict()
dts = [(x[2] - x[1]) / T_final for x in zip(coarse_times[1:end-1], coarse_times[2:end])]
for spin_idx in 1:N-1
	areas[spin_idx] = sum(dts .* magnetizations[spin_idx, 2:end]) |> abs

	# Edwards-Anderson
	EA_param[spin_idx] = (1/(N-1)) * sum(dts .* nz_coarse[spin_idx, 2:end] .^ 2) |> abs
end

top_idxs = [k for (k, v) in sort(areas |> collect, by = x -> x[2])]
top_idxs = [k for (k, v) in sort(EA_param |> collect, by = x -> x[2])]
top_idx = top_idxs[1]

In [ ]:
regular_trajectories = filter!(x -> x != top_idx, collect(1:N-1));

In [ ]:
figure(figsize = (4, 5))

c1 = "-C1"
c2 = "-C0"

ax = subplot(411)
axvline(gaploc, ls = "-", c="k", alpha=0.2, lw=6)
for spin_nr in regular_trajectories
	plot(sol_t ./ sol_t[end], nx_vals[spin_nr, :], "-k", lw=0.75)
end
plot(sol_t ./ sol_t[end], nx_vals[top_idx, :], c1, lw=1.5, label = @sprintf("\$i=%i\$", top_idx))
xlim(0, 1)
ax.set_xticklabels([])
ylim(-0.1, 1.1)
ylabel("\$n_i^x(s)\$")
legend(handlelength=1.2, handletextpad=0.5, borderaxespad=0.2)

ax = subplot(412)
axvline(gaploc, ls = "-", c="k", alpha=0.2, lw=6)
for spin_nr in regular_trajectories
	plot(sol_t ./ T_final, nz_vals[spin_nr, :], "-k", lw=0.75)
end
plot(sol_t ./ T_final, nz_vals[top_idx, :], c1, lw=1.5)

# Edwards-Anderson
plot(coarse_times, sum([nz_coarse[spin_nr, :] .^ 2 for spin_nr in 1:N-1]) ./ (N-1), "-C0", lw=2.5, label="\$q_{\\mathrm{EA}}\$")
xlim(0, 1)
ylim(-1.1, 1.1)
ax.set_xticklabels([])
ylabel("\$n_i^z(s)\$")
legend(handlelength=1.2, handletextpad=0.5, borderaxespad=0.2)

ax = subplot(413)
axvline(gaploc, ls = "-", c="k", alpha=0.2, lw=6)
for spin_nr in regular_trajectories
	plot(coarse_times, (abs.(complex_coordinate(spin_nr, nx_coarse, ny_coarse, nz_coarse))), "-k", lw=0.75)
end
plot(coarse_times, (abs.(complex_coordinate(top_idx, nx_coarse, ny_coarse, nz_coarse))), c1, lw=1.5, label = @sprintf("\$i=%i\$", top_idx))

xlim(0, 1)
ylim(0, 1.5)
ax.set_xticklabels([])
ylabel("\$|z_i(s)|\$")

ax = subplot(414)
axvline(gaploc, ls = "-", c="k", alpha=0.2, lw=6)
for spin_nr in regular_trajectories
	plot(coarse_times, magnetizations[spin_nr, :], "-k", lw=0.75)
end
plot(coarse_times, magnetizations[top_idx, :], c1, lw=1.5, label = @sprintf("\$i=%i\$", top_idx))

xlim(0.0, 1)
ylim(-3, 3)
xlabel("\$s\$")
ylabel("\$m_i(s)\$")

tight_layout(pad=0.1)
display(gcf())
savefig(PLOT_PATH * @sprintf("frustrated_ring_mean_fields_N_%i.pdf", N-1), dpi=256, bbox_inches="tight")
PythonPlot.close();

## Energies

In [ ]:
sigma_star = sign.(sol_u[end, 3, :])
h = mf_problem.local_fields
J = mf_problem.couplings
E = sum([-h[l] * sigma_star[l] for l in 1:N-1]) + sum([-J[i, j] * sigma_star[i] * sigma_star[j] for i in 1:N-1 for j in (i+1):N-1])

In [ ]:
E_0 = -(N - 3) + J_R - 2J_L
E_1 = E_0 + 2(J_L - J_R)

E_0 |> println
E_1 |> println

In [ ]:
[λ[k, :][end] for k in 1:6] |> println

In [ ]:
isapprox.([λ[k, :][end] for k in 1:6], E, atol=1e-5) |> println

## Statistical Green function

In [ ]:
# statistical Green function
coarse_times = range(0, 1, npts + 1)
lyapunov_parameters = LyapunovParameters(T_final, npts, tol, tol)
mf_sol, stat_GF = statistical_green_function(mf_problem, lyapunov_parameters)

flucs = k -> (real.(1.0im .* diag(stat_GF[k])[1:mf_problem.num_qubits]) .- 1.0) ./ 2;
all_flucs = reduce(hcat, map(flucs, 1:npts+1));

In [ ]:
regular_trajectories = filter!(x -> x != top_idxs[1], collect(1:N-1));

scale_factors = [1 .+ abs.(complex_coordinate(i, nx_coarse, ny_coarse, nz_coarse)) .^ 2 for i in 1:N-1]
mean_flucs = mean([smoothen(all_flucs[i, :], coarse_times) for i in regular_trajectories], dims=1)[1];
mean_scaled_flucs = mean([smoothen(scale_factors[i] .^ 2 .* all_flucs[i, :], coarse_times) for i in regular_trajectories], dims = 1)[1];

In [ ]:
local_EA_param = 1. # nz_coarse[top_idx, :].^2
q_EA = sum([nz_coarse[spin_nr, :] .^ 2 for spin_nr in 1:N-1]) ./ (N-1)
scale_factor = 1 .+ abs.(complex_coordinate(top_idx, nx_coarse, ny_coarse, nz_coarse)) .^ 2

top_flucs = smoothen(scale_factor.^2 .* all_flucs[top_idx, :], coarse_times, navg=256)
# crit_flucs = smoothen(scale_factor.^2 .* all_flucs[top_idx, :] .* local_EA_param, coarse_times, navg=256);
crit_flucs = smoothen(scale_factor.^2 .* all_flucs[top_idx, :] .* q_EA, coarse_times, navg=256);

In [ ]:
maximum(abs.(overlap_01_z .- overlap_01_x) ./ ((λ[2, :] .- λ[1, :]).^2))

In [ ]:
figure(figsize = (5., 5.4))
ax = subplot(211)
twinax = ax.twinx()
# axvline(gaploc, ls = "-", c="k", alpha=0.2, lw=6)
ax.semilogy(exact_times, abs.(overlap_02_z .- overlap_02_x) ./ ((λ[3, :] .- λ[1, :]).^2), "-C0", label="\$\\alpha = 2\$", lw=1.25)
# ax.semilogy(exact_times, abs.(overlap_01_z .- overlap_01_x) ./ ((λ[2, :] .- λ[1, :]).^2), "C0", label = "\$\\alpha = 1\$", lw=2)
twinax.plot(coarse_times, crit_flucs, c1, lw=1.5, label=@sprintf("\$i_*=%s\$", top_idx))
xlim(0.0, 1.0)
ax.set_ylim(1e0, 1e2)
ax.set_yticks([1e0, 1e1, 1e2])
twinax.set_ylim(0, 0.1)
ax.tick_params(axis="y", which="both", colors="C0")
twinax.spines["left"].set_color("C0")
twinax.spines["right"].set_color("C1")
twinax.tick_params(axis="y", which="both", colors="C1")
ax.set_xlabel("\$s\$")
# ylabel("\$\\frac{|\\langle \\alpha|H_Z-H_X|0\\rangle|}{E_\\alpha - E_0}\$")
ax.set_ylabel("\$|\\langle \\alpha|\\hat H_Z \\hspace{-1mm} - \\hspace{-0.5mm} \\hat H_X|0\\rangle|/(E_\\alpha \\hspace{-1mm} - \\hspace{-0.5mm} E_0)^2\$")
twinax.set_ylabel("\$ q_{\\mathrm{EA}}(s) \\langle\\delta  z_{i_*}(s) \\delta \\bar z_{i_*}(s) \\rangle\$")
ax.legend(loc="upper left", handlelength=1.2, handletextpad=0.5, borderaxespad=0.3)
twinax.legend(loc="upper right", frameon=false, framealpha=0.9, edgecolor="w", 
              handlelength=1.2, handletextpad=0.5, borderaxespad=0.3)

tight_layout(pad=0.2)
savefig(PLOT_PATH * @sprintf("frustrated_ring_metric_N_%i.pdf", N-1), dpi=256, bbox_inches="tight")
display(gcf())
PythonPlot.close();

## Fluctuations

In [ ]:
figure(figsize = (7.5, 2.5))

ax = subplot(121)
# axvline(gaploc, ls = "-", c="k", alpha=0.2, lw=6)
all_colors = ["-k" for _ in 1:N-1]
all_colors[top_idx] = c1
all_labels = ["" for _ in 1:N-1]
all_labels[top_idx] = @sprintf("\$i=%s\$", string(top_idx))
for i in 1:(N-1)
	scale_factor = 1 .+ abs.(complex_coordinate(i, nx_coarse, ny_coarse, nz_coarse)) .^ 2
	plot(coarse_times,  smoothen(scale_factor .^ 2 .* all_flucs[i, :] .* q_EA, coarse_times, navg=512), label=all_labels[i], all_colors[i])
	xlim(0.0, 1.0)
end
legend(frameon = false, handlelength = 1, ncol=1)
xlabel("\$s\$")
# ax.set_ylabel("\$ -\\mathrm{Im}\\, G^<_{ii}(s, s) \$")
ax.set_ylabel("\$ q_{\\mathrm{EA}}(s) \\langle\\delta  z_{i_*}(s) \\delta \\bar z_{i_*}(s) \\rangle\$")
xlim(0, 1)
ylim(0, 0.1)

tight_layout(pad=0.1)
savefig(PLOT_PATH * @sprintf("frustrated_ring_statistical_N_%i.pdf", N - 1), dpi=256, bbox_inches="tight")
display(gcf())
PythonPlot.close();

## Testing

In [ ]:
figure(figsize = (4, 4))

ax = subplot(211)

axvline(gaploc, ls = "-", c="k", alpha=0.2, lw=6)

# ax.semilogy(exact_times, all_data[13]["g_alphas"][1], label="\$\\alpha = 1\$", "-C0", lw=2.0)
# ax.semilogy(exact_times, all_data[15]["g_alphas"][1], label="\$\\alpha = 1\$", "-C0", lw=2.0)
ax.semilogy(exact_times, all_data[13]["g_alphas"][2], label=@sprintf("\$N=%i\$", 12), "-.C0", lw=1.25)
ax.semilogy(exact_times, all_data[15]["g_alphas"][2], label=@sprintf("\$N=%i\$", 14), "--C1", lw=1.25)
ax.semilogy(exact_times, all_data[17]["g_alphas"][2], label=@sprintf("\$N=%i\$", 16), "-C2", lw=1.25)


ax.set_xlim(0.0, 1.0)
# ax.set_ylim(1e-1, 1e5)
ax.set_ylim(1e-1, 1e2)
# ax.set_ylim(0, 50)
# ax.set_yticks([0, 25, 50])
ax.set_xlabel("\$s\$")
ax.set_ylabel("\$ g_2(s)\$")
ax.legend(loc="upper left", handlelength=1.2, handletextpad=0.5, borderaxespad=0.3)

tight_layout(pad=0.25)
savefig(PLOT_PATH * @sprintf("frustrated_ring_g_alphas.pdf"), dpi=256, bbox_inches="tight")
display(gcf())
PythonPlot.close();

In [ ]:
figure(figsize = (4, 3))

c1 = "-C1"
c2 = "-C0"

ax = subplot(211)

axvline(gaploc, ls = "-", c="k", alpha=0.2, lw=6)
# for spin_nr in regular_trajectories
# 	plot(sol_t ./ T_final, nz_vals[spin_nr, :], "-k", lw=0.75)
# end
semilogy(coarse_times, smoothen(nz_coarse[top_idx, :], coarse_times, navg=256), c1, lw=1.5)
ylim(1e-7, 1e-2)

ax = subplot(212)
axvline(gaploc, ls = "-", c="k", alpha=0.2, lw=6)
for spin_nr in regular_trajectories
	plot(coarse_times, magnetizations[spin_nr, :], "-k", lw=0.75)
end
plot(coarse_times, magnetizations[top_idx, :], c1, lw=1.5, label = @sprintf("\$i=%i\$", top_idx))

xlim(0.0, 1)
ylim(-3, 3)
xlabel("\$s\$")
ylabel("\$m_i(s)\$")

tight_layout(pad=0.1)
display(gcf())
# savefig(PLOT_PATH * @sprintf("frustrated_ring_mean_fields_N_%i.pdf", N-1), dpi=256, bbox_inches="tight")
PythonPlot.close();

In [ ]:
# figure(figsize = (4, 5))

# ax = subplot(311)
# axvline(gaploc, ls = "-", c="k", alpha=0.2, lw=6)
# for i in 1:5
# 	# if i == 2 
# 	# 	plot(exact_times, (λ[i, :] .- λ[1, :]), lw = 2.0, alpha = 1)
# 	# elseif i == 3		
# 	# 	plot(exact_times, (λ[i, :] .- λ[1, :]), lw = 2.0, alpha = 1)	
# 	if i == 3 
# 		plot(exact_times, (λ[i, :] .- λ[1, :]), lw = 2.0, alpha = 1)
# 	elseif i == 4		
# 		plot(exact_times, (λ[i, :] .- λ[1, :]), lw = 2.0, alpha = 1)
# 	else
# 		plot(exact_times, (λ[i, :] .- λ[1, :]), "-k", lw=0.75, alpha = 1)
# 	end
# end
# plot([], [], "-k", lw = 1, label = @sprintf("\$N=%i\$", N-1))
# # plot([], [], "-k", lw = 1, label = @sprintf("\$N=%i\$", N))
# xlim(0.0, 1.0)
# ax.set_yticks(2 .* [0, 0.5, 1.0])
# ylim(0, 2)
# ax.set_xticklabels([])
# ylabel("\$E_\\alpha - E_0\$")
# legend(frameon = false, handlelength=1., handletextpad=0.5, borderaxespad=0.2)

# ax = subplot(312)
# axvline(gaploc, ls = "-", c="k", alpha=0.2, lw=6)
# plot(exact_times, abs.(overlap_03_z .- overlap_03_x), label = "\$\\alpha = 3\$", "-C1", lw=1.25)
# plot(exact_times, abs.(overlap_02_z .- overlap_02_x), label = "\$\\alpha = 2\$", "-C0", lw=1.25)

# # plot(exact_times, abs.(overlap_02_z .- overlap_02_x), label = "\$\\alpha = 2\$", "-C1", lw=1.25)
# # plot(exact_times, abs.(overlap_01_z .- overlap_01_x), label = "\$\\alpha = 1\$", "-C0", lw=1.0)

# plot(exact_times, abs.(overlap_13_z .- overlap_13_x), "--C1", lw=1.25)
# plot(exact_times, abs.(overlap_12_z .- overlap_12_x), "--C0", lw=1.25)

# xlim(0.0, 1.0)
# ax.set_xticklabels([])
# # ax.set_yticks([0, 1.0])
# ylim(0, 1)
# # ylim(0, 2)
# # xlabel("\$s\$")
# ylabel("\${|\\langle \\alpha|\\hat H_Z- \\hat H_X|0\\rangle|}\$")

# legend(ncol=1, handlelength=1., handletextpad=0.5, borderaxespad=0.2, loc="upper left")

# ax = subplot(313)
# axvline(gaploc, ls = "-", c="k", alpha=0.2, lw=6)
# ax.semilogy(exact_times, abs.(overlap_03_z .- overlap_03_x) ./ ((λ[4, :] .- λ[1, :]).^2), label="\$\\alpha = 3\$", "-C1", lw=1.25)
# ax.semilogy(exact_times, abs.(overlap_02_z .- overlap_02_x) ./ ((λ[3, :] .- λ[1, :]).^2), label="\$\\alpha = 2\$", "-C0", lw=1.25)

# ax.plot(exact_times, abs.(overlap_13_z .- overlap_13_x) ./ ((λ[4, :] .- λ[1, :]).^2), "--C1", lw=1.25)
# ax.plot(exact_times, abs.(overlap_12_z .- overlap_12_x) ./ ((λ[3, :] .- λ[1, :]).^2), "--C0", lw=1.25)

# ax.set_xlim(0.0, 1.0)
# # ax.set_ylim(1e-1, 1e5)
# ax.set_ylim(1e-1, 1e8)
# # ax.set_ylim(0, 50)
# # ax.set_yticks([0, 25, 50])
# ax.set_xlabel("\$s\$")
# ax.set_ylabel("\$ g_\\alpha(s)\$")
# ax.legend(loc="upper left", handlelength=1.2, handletextpad=0.5, borderaxespad=0.3)

# tight_layout(pad=0.25)
# # savefig(PLOT_PATH * @sprintf("frustrated_ring_exact_spectrum_N_%i.pdf", N-1), dpi=256, bbox_inches="tight")
# # savefig(PLOT_PATH * @sprintf("frustrated_ring_exact_spectrum_N_%i.pdf", N), dpi=256, bbox_inches="tight")
# display(gcf())
# PythonPlot.close();